In [1]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
import warnings

from cuml.cluster import KMeans as KMeans
warnings.simplefilter('ignore')

In [2]:
from external_lib.deep_sort_pytorch.utils.parser import get_config
from external_lib.deep_sort_pytorch.deep_sort import DeepSort

from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features


In [3]:
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"
debug = True

## function

In [4]:
def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label: 
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im



In [5]:



def deepsort_helmets(video_data,
                     video_dir,
                     deepsort_config='deepsort.yaml',
                     plot=False,
                     plot_frames=[]):
    
    # Setup Deepsort
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,
                        max_dist=cfg.DEEPSORT.MAX_DIST,
                        min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                        max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,
                        nn_budget=cfg.DEEPSORT.NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True)
    ds = []
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):
        d['x'] = (d['left'] + round(d['width'] / 2))
        d['y'] = (d['top'] + round(d['height'] / 2))

        xywhs = d[['x','y','width','height']].values

        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        
        ##シーケンス
        success, image = cap.read()
        
        # 画像の色の順番を変更（BGR to RGB）
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        confs = np.ones([len(d),])
        clss =  np.zeros([len(d),])
        
        outputs = deepsort.update(xywhs, confs, clss, image)
        
        if (plot and frame > cfg.DEEPSORT.N_INIT) or (frame in plot_frames):
            for j, (output, conf) in enumerate(zip(outputs, confs)): 

                bboxes = output[0:4]
                id = output[4]
                cls = output[5]

                c = int(cls)  # integer class
                label = f'{id}'
                color = compute_color_for_id(id)
                im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
            fig, ax = plt.subplots(figsize=(15, 10))
            video_frame = d['video_frame'].values[0]
            ax.set_title(f'Deepsort labels: {video_frame}')
            plt.imshow(im)
            plt.show()

        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom','deepsort_cluster','class'])
            
        if len(preds_df) > 0:
            # TODO Fix this messy merge
            d = pd.merge_asof(d.sort_values(['left','top']),
                              preds_df[['left','top','deepsort_cluster']] \
                              .sort_values(['left','top']), on='left', suffixes=('','_deepsort'),
                              direction='nearest')
        ds.append(d)
    dout = pd.concat(ds)
    return dout


def add_deepsort_label_col(out):
    # Find the top occuring label for each deepsort_cluster
    sortlabel_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label_count'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map)
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map)

    return out


def score_vs_deepsort(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_deepsort':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} deepsort')

In [6]:
from lib.noglobal import noglobal

submission_df_v2 = pd.read_csv("/work/data/input/nfl-health-and-safety-helmet-assignment/train_baseline_helmets.csv")
myvideo = "57778_004244_Endzone"
sub  = submission_df_v2[submission_df_v2["video_frame"].str.contains(myvideo)].reset_index()
sub['frame'] = sub['video_frame'].str.split('_').str[-1].astype('int')
sub = sub[sub["conf"] > 0.2]

#myvideo = "57596_002686"

labels = pd.read_csv(f"{base_dir}/train_labels.csv")

In [7]:
# Add video and frame columns to submission.
submission_df = sub;

submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_')
submission_df['frame'] = submission_df['video_frame'].str.split('_').str[-1].astype('int')



if debug:
    video_dir = f"{base_dir}/train/"
else:
    video_dir = f"{base_dir}/test/"

    
    

# Loop through test videos and apply. If in debug mode show the score change.
outs = []
for myvideo, video_data in tqdm(submission_df.groupby('video'), total=submission_df['video'].nunique()):
    #print(myvideo)
    #print(f'==== {myvideo} ====')
    
    
    if debug:
        out = deepsort_helmets(video_data, video_dir)        
    else:
        out = deepsort_helmets(video_data, video_dir)        
    #out = add_deepsort_label_col(out)
    outs.append(out)
    if False:        
        score_vs_deepsort(myvideo, out, labels)
        
    
submission_deepsort = pd.concat(outs).copy()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

## 画素値集計

In [8]:
@noglobal()
def plot(image,each_df):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    each_df["right"] = each_df["left"] + each_df["width"]
    each_df["bottom"] = each_df["top"] + each_df["height"]
    
    outputs = each_df[["left","top","right","bottom","deepsort_cluster"]].values
    for j, output in enumerate(outputs): 
        bboxes = output[0:4]
        id = output[4]
        
        label = f'{id}'
        color = compute_color_for_id(id)
        im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
    fig, ax = plt.subplots(figsize=(15, 10))
    video_frame = each_df['video_frame'].values[0]
    ax.set_title(f'Deepsort labels: {video_frame}')
    plt.imshow(im)
    plt.show()
    
    
@noglobal()
def extract_pixel(img,box):
    #left = box[0]
    #right = box[2]
    
    #upper = box[1]
    #lower = box[3]
    
    left = int(box[0] + (box[2] - box[0])*2/6)
    right = int(box[0] + (box[2] - box[0])*4/6)
    
    upper = int(box[1] + (box[3] - box[1])*2/6)
    lower = int(box[1] + (box[3] - box[1])*4/6)
    
    ret_list = []
    for x in range(left,right):
        for y in range(upper,lower):
            ret_list.append(img[y,x])
            
    
    return np.mean(ret_list,axis=0).tolist()


@noglobal()
def add_team_speculation_(submission_deepsort,video_dir = video_dir,plot=False):
    submission_deepsort["right"] = submission_deepsort["left"] + submission_deepsort["width"]
    submission_deepsort["bottom"] = submission_deepsort["top"] + submission_deepsort["height"]
    
        
    video_df_list = []
    
    
    for myvideo, each_df in tqdm(submission_deepsort.groupby("video")):                        
        
        print(f'{video_dir}/{myvideo}.mp4')
        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
        
        each_frame_df_list = [];
        
        for frame, d in tqdm(each_df.groupby(["frame"])):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
            ##シーケンス
            success, image = cap.read()
            

            # 画像の色の順番を変更（BGR to RGB）
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            
            pixel_value_list = [];
            for index,row in d.iterrows():
                bboxes = row[["left","top","right","bottom"]].values                
                pixel_value_list.append(extract_pixel(image,bboxes))
                
            
            pixel_value_list_numpy = np.array(pixel_value_list)
            d["average_pixel_r"] = pixel_value_list_numpy[:,0]
            d["average_pixel_g"] = pixel_value_list_numpy[:,1]
            d["average_pixel_b"] = pixel_value_list_numpy[:,2]
            d["frame"] = frame
            
            each_frame_df_list.append(d);
            
        all_frame_df = pd.concat(each_frame_df_list);
                                
        kmeans = KMeans(n_clusters=2, max_iter=100000, init='scalable-k-means++')
        pixel_value_list_numpy = all_frame_df[["average_pixel_r","average_pixel_g","average_pixel_b"]].values
        b = kmeans.fit(pixel_value_list_numpy)        
        all_frame_df["team"] = kmeans.labels_
                
        video_df_list.append(all_frame_df)
        
        
    return pd.concat(video_df_list)
    if (False):
        for frame, d in tqdm(video_df_list[0].groupby(['frame']), total=video_df_list[0]['frame'].nunique()):
    
            xywhs = d[['left','top','right','bottom']].values
            labels = d["team"].values

            
            cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional

            ##シーケンス
            success, image = cap.read()

            # 画像の色の順番を変更（BGR to RGB）
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            cfg = get_config()
            cfg.merge_from_file("deepsort.yaml")    
            if (True and frame > cfg.DEEPSORT.N_INIT):

                
                
                for j, (output, label) in enumerate(zip(xywhs, labels)): 

                    bboxes = output[0:4]
                    label = f'{label}'

                    color = compute_color_for_id(int(label))
                    if (label == "0"):
                        im = plot_one_box(bboxes, image, label=str(label), color=[0,255,0], line_thickness=2)    
                    elif (label == "1"):
                        im = plot_one_box(bboxes, image, label=str(label), color=[255,0,0], line_thickness=2)    
                    else:
                        raise Exception("asdf");



                ig, ax = plt.subplots(figsize=(15, 10))
                video_frame = d['frame'].values[0]
                ax.set_title(f'Deepsort labels: {video_frame}')
                plt.imshow(im)
                plt.show()


            
            
            
            
            
            
            
    
    
sub_df = add_team_speculation_(submission_deepsort.copy(),plot=True)

    

  0%|          | 0/1 [00:00<?, ?it/s]

/work/data/input/nfl-health-and-safety-helmet-assignment/train//57778_004244_Endzone.mp4


  0%|          | 0/473 [00:00<?, ?it/s]

In [35]:
sub_df.to_csv("deepsort_result.csv",index=False)

In [10]:
nOf_variable = target_df["deepsort_cluster"].nunique()

In [33]:
preprocess_tracking(target_video)




,index,gameKey,playID,player,time,x,y,s,a,dis,o,dir,event,game_play,snap,isSnap,team,snap_offset,est_frame
0,111028,57778,4244,H69,2018-12-16 21:02:49.099000+00:00,19.94,28.38,0.01,0.31,0.00,61.86,129.84,line_set,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Home,-0.200,-2
1,111029,57778,4244,H69,2018-12-16 21:02:49.200000+00:00,19.96,28.36,0.08,0.70,0.02,63.85,135.07,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Home,-0.099,4
2,111030,57778,4244,H69,2018-12-16 21:02:49.299000+00:00,19.98,28.33,0.20,1.10,0.04,65.42,141.58,ball_snap,57778_004244,2018-12-16 21:02:49.299000+00:00,True,Home,0.000,10
3,111031,57778,4244,H69,2018-12-16 21:02:49.400000+00:00,20.03,28.26,0.42,1.57,0.09,66.06,145.39,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Home,0.101,16
4,111032,57778,4244,H69,2018-12-16 21:02:49.500000+00:00,20.08,28.16,0.64,1.77,0.11,65.35,148.33,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Home,0.201,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,116026,57778,4244,V94,2018-12-16 21:02:59.599000+00:00,24.32,25.76,0.27,0.71,0.04,104.47,1.74,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Away,10.300,627
2416,116027,57778,4244,V94,2018-12-16 21:02:59.700000+00:00,24.32,25.81,0.35,0.66,0.05,105.18,2.49,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Away,10.401,633
2417,116028,57778,4244,V94,2018-12-16 21:02:59.799000+00:00,24.32,25.86,0.40,0.49,0.05,106.09,2.49,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Away,10.500,639
2418,116029,57778,4244,V94,2018-12-16 21:02:59.900000+00:00,24.33,25.92,0.46,0.41,0.06,103.15,2.54,NaN,57778_004244,2018-12-16 21:02:49.299000+00:00,False,Away,10.601,645


In [ ]:
before_df = None;
for frame,each_df in submission_deepsort.groupby("frame"):    
    
        
    if ( ( before_df is None) or ( before_df["deepsort_cluster"].isnull().sum() == before_df.shape[0] ) ):
        before_df = each_df
        continue;
    
    
    
    if (frame < 20):    
        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        
        ##シーケンス
        success, image = cap.read()
        
        
        plot(image,each_df)
        display(each_df.shape[0],each_df["deepsort_cluster"].nunique())
        display(each_df)
        display(before_df)
        
    #break;
    
    